In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

### Make scene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.panda, None,
                INDY_IP)]
              , connection_list=[False])
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"panda0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


## Preapare background

In [3]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.3,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="panda0_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
grip0 = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
gscene.show_pose(crob.home_pose)

Please create a subscriber to the marker


## Door

In [4]:
from pkg.utils.code_scraps import *

In [5]:
from pkg.planning.constraint.constraint_actor import KnobFramer, HingeFramer
from pkg.planning.constraint.constraint_subject import KnobFrame, HingeFrame

##
# @class KnobTask
# @brief sweep action points in alphabetical order
# @remark   state_param: boolean vector of which each element represents if each waypoint is covered or not
#           node_item: number of covered waypoints
class KnobTask(SweepTask):
    def __init__(self, oname, geometry, lever, action_points_dict, sub_binders_dict=None, tol=1e-3):
        self.lever = lever
        self.T0 = geometry.Toff
        self.link0 = geometry.link_name
        SweepTask.__init__(self, oname, geometry, action_points_dict, 
                           sub_binders_dict=sub_binders_dict, tol=tol, one_direction=False)
    
    ##
    # @brief set object binding state and update location
    # @param binding BindingChain
    # @param state_param list of done-mask
    def set_state(self, binding, state_param=None):
        self.binding = deepcopy(binding)
        self.state_param = np.zeros(len(self.action_points_order), dtype=np.bool)
        if self.binding[1] is not None:
            if self.binding[1] in self.action_points_order:
                self.state_param[:self.action_points_order.index(self.binding[1])+1] = True
            else:
                raise(RuntimeError("Undefined handle {}".format(self.binding[1])))
        if binding.handle_name is None:
            handle = self.action_points_dict[self.action_points_order[0]]
        else:
            handle = self.action_points_dict[binding.handle_name]
        actor = self.lever
        btf = BindingTransform(self, handle, actor)
        if all(self.state_param):
            raise("previous state not guaranteed")
            Thg = np.matmul(SE3_inv(handle.Toff_lh), self.geometry.Toff)
            Tlag = matmul_series(btf.T_actor_lh, SE3_inv(btf.T_add_handle), Thg)
            if (np.linalg.norm(Tlag-self.geometry.Toff)>1e-5 
                or self.geometry.link_name != actor.geometry.link_name):
                self.geometry.set_offset_tf(Tlag[:3,3], Tlag[:3,:3])
                self.geometry.set_link(actor.geometry.link_name)
        else:
            self.geometry.set_offset_tf(self.T0[:3,3], self.T0[:3,:3])
            self.geometry.set_link(self.link0)
            
        self.update_sub_points()

    ##
    # @brief make constraints. by default, empty list.
    # @remark constraint is applied when using same binding
    # @param binding_from previous binding
    # @param binding_to next binding
    def make_constraints(self, binding_from, binding_to, tol=None):
        if binding_from is not None and binding_from.actor_name == binding_to.actor_name:
            return "Constraint not implemented yet. Use MoveitPlanner.incremental_constraint_motion=True"
        else:
            return []
        
    ##
    # @brief get object-level neighbor component (detach or next waypoint)
    def get_neighbor_node_component_list(self, node_tem, pscene):
        if node_tem == len(self.state_param):
            neighbor = [node_tem]
        else:# node_tem < len(self.state_param):
            neighbor = [node_tem + 1]
        if (not self.one_direction) and node_tem > 0:
            neighbor.insert(0, node_tem - 1)
        return neighbor
        

```python
BindingChain(subject_name='knob', handle_name='r2', actor_name='knob_plug', actor_root_gname='lever')
```

In [6]:
def add_door(pscene, dname = "door", center = (0.5,0,0.5), rpy = (0,0,0),
             dims = (0.05, 0.4, 0.5), hinge_point = (-0.025, 0.2, 0),
             door_ang = np.pi*1/4, door_div = 1, link_name="base_link", clearance=3e-3,
             frame_depth=0.0, frame_thickness=0.01, color = (0.8,0.8,0.8,1)):
    gscene = pscene.gscene
    door = gscene.create_safe(GEOTYPE.BOX, dname, link_name=link_name,
                              center=center, rpy=rpy, dims=dims,
                              fixed=False, collision=True, color=color)
    door_frame = gscene.create_safe(GEOTYPE.BOX, dname+"_frame", link_name=link_name,
                                    center=center, rpy=Rot2rpy(np.matmul(Rot_rpy(rpy), Rot_axis(2,np.pi/2))),
                                    dims=np.asarray(dims)[[2,1,0]],
                                    fixed=True, collision=False, color=(1,1,1,0.0), display=False)
#     gscene.add_virtual_guardrail(door_frame, axis="xy", color=color, THICKNESS=frame_thickness,
#                                  HEIGHT=dims[0]/2+frame_depth, margin=frame_thickness/2+clearance)

    door_r = pscene.create_binder(bname=door.name+"_hinge", gname=door.name, _type=HingeFramer,
                                  point=hinge_point)

    hinge_points = []
    for i_div in range(door_div+1):
        ang = door_ang/door_div*i_div
        hinge_points.append(HingeFrame("h{}".format(i_div), door_frame,
                                       np.asarray(hinge_point)[[2,1,0]], 
                                       Rot2rpy(
                                           np.matmul(Rot_axis(2,np.pi/2).transpose(), 
                                                     Rot_axis(3, -ang)))))

    door_s = pscene.create_subject(oname=door.name, gname=door_frame.name, _type=SweepTask,
                                  action_points_dict={hp.name: hp for hp in hinge_points},
                                  one_direction=False)
    return door, door_s

In [7]:

def add_lever(pscene, knob,lname="lever", lever_ang=np.pi/4, knob_offset=(0.09), dims=(0.02, 0.2, 0.02),
              link_name="base_link", color=(0.8, 0.8, 0.8, 1), clearance = 1e-3):
    gscene = pscene.gscene
    Q0 = [0]*gscene.joint_num
    Tbk = knob.get_tf(Q0)
    Tbkp = np.matmul(Tbk, SE3(np.identity(3), (0, 0, knob.dims[2]/2+clearance)))
    Tbl = np.matmul(Tbkp, SE3(np.identity(3), (0, knob_offset, dims[2]/2)))
    lever = gscene.create_safe(GEOTYPE.BOX, lname, link_name=link_name,
                                    center=Tbl[:3,3], rpy=Rot2rpy(Tbl[:3,:3]), dims=dims,
                                    fixed=False, collision=True, color=color)
    lgrasp = Grasp2Point("gp", lever, (0, 0, 0), (np.pi/2, 0, -np.pi/2))
    lattach = PlaceFrame("cp", lever, (0, -knob_offset, -dims[2]/2), (0,0,0))

    lever_g = pscene.create_subject(oname=lname+"_grip", gname=lever.name, _type=CustomObject,
                                     action_points_dict={lgrasp.name: lgrasp, lattach.name: lattach})
    kpoint0 = pscene.create_binder(bname=knob.name + "_0", gname=knob.name, _type=AttachFrame,
                                   point=(0, 0, knob.dims[2]/2+clearance))
    kpoint1 = pscene.create_binder(bname=knob.name + "_1", gname=knob.name, _type=AttachFrame,
                                   point=(0, 0, knob.dims[2]/2+clearance), rpy=(0, 0, lever_ang))

    lever_plug = pscene.create_binder(bname=knob.name+"_plug", gname=lname, _type=KnobFramer,
                                   point=(0, -knob_offset, -dims[2]/2))
    rp1 = KnobFrame("r1", knob, (0, 0, knob.dims[2]/2+clearance), (0,0,0))
    rp2 = KnobFrame("r2", knob, (0, 0, knob.dims[2]/2+clearance), rpy=(0, 0, lever_ang))

    knob_s = pscene.create_subject(oname=knob.name, gname=knob.name, _type=KnobTask,
                                   action_points_dict={rp1.name: rp1, rp2.name: rp2}, 
                                   lever = lever_plug)
    return knob_s

In [8]:
door, door_s = add_door(pscene, center=(0.7,-0.2,0.5), color=(0.8,0.4,0.2,1))
knob = gscene.create_safe(GEOTYPE.BOX, 'knob', link_name="base_link",
                          center=(-door.dims[0]/2-0.01,-door.dims[1]/2+0.05,0), rpy=(0,-np.pi/2,0), dims=(0.02,0.02,0.02),
                          fixed=False, collision=True, color=(0.8,0.8,0.8,1), parent="door")

knob_s = add_lever(pscene, knob, dims=(0.02,0.1,0.02), knob_offset=0.04)

### prepare planner

In [9]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


In [10]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan.motion_filters = [GraspChecker(pscene), ReachChecker(pscene)]
mplan.incremental_constraint_motion = True

   Use a production WSGI server instead.
 * Debug mode: off


### Plan

In [11]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

['door', 'knob', 'lever_grip']
(0, 0, 'knob')


In [12]:
goal_nodes = [(2, 2,'grip0')]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=False)

try: 0 - (0, 0, 'knob')->(0, 1, 'knob')
result: 0 - (0, 0, 'knob')->(0, 1, 'knob') = fail
try: 0 - (0, 0, 'knob')->(1, 0, 'knob')
result: 0 - (0, 0, 'knob')->(1, 0, 'knob') = fail
try: 0 - (0, 0, 'knob')->(0, 0, 'grip0')
try transition motion
transition motion tried: True
result: 0 - (0, 0, 'knob')->(0, 0, 'grip0') = success
branching: 0->1 (0.08/30.0 s, steps/err: 49(55.7250976562 ms)/0.00117738217944)
try: 1 - (0, 0, 'grip0')->(1, 0, 'grip0')
result: 1 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
try: 1 - (0, 0, 'grip0')->(0, 0, 'knob')
result: 1 - (0, 0, 'grip0')->(0, 0, 'knob') = success
branching: 1->2 (0.09/30.0 s, steps/err: 1(4.01210784912 ms)/0)
try: 2 - (0, 0, 'knob')->(1, 0, 'knob')
result: 2 - (0, 0, 'knob')->(1, 0, 'knob') = fail
try: 1 - (0, 0, 'grip0')->(0, 1, 'grip0')
result: 1 - (0, 0, 'grip0')->(0, 1, 'grip0') = success
branching: 1->3 (0.11/30.0 s, steps/err: 1(10.4219913483 ms)/0)
Not available:knob-0
np_exclude:['r1']
bd_exclude:knob_plug
============== Non-available 

transition motion tried: True
result: 8 - (0, 0, 'knob')->(0, 0, 'grip0') = success
branching: 8->13 (4.06/30.0 s, steps/err: 18(165.926218033 ms)/0.00197550329853)
try: 13 - (0, 0, 'grip0')->(1, 0, 'grip0')
result: 13 - (0, 0, 'grip0')->(1, 0, 'grip0') = fail
try: 5 - (0, 0, 'knob')->(0, 0, 'grip0')
try transition motion
transition motion tried: True
result: 5 - (0, 0, 'knob')->(0, 0, 'grip0') = success
branching: 5->14 (4.15/30.0 s, steps/err: 15(72.9570388794 ms)/0.00160430134063)
try: 14 - (0, 0, 'grip0')->(0, 1, 'grip0')
try transition motion
transition motion tried: True
result: 14 - (0, 0, 'grip0')->(0, 1, 'grip0') = success
branching: 14->15 (4.19/30.0 s, steps/err: 8(42.7870750427 ms)/0.00164484676561)
try: 11 - (0, 2, 'grip0')->(1, 2, 'grip0')
try transition motion
transition motion tried: True
result: 11 - (0, 2, 'grip0')->(1, 2, 'grip0') = success
branching: 11->16 (4.23/30.0 s, steps/err: 15(37.7728939056 ms)/0.00128433711082)
try: 16 - (1, 2, 'grip0')->(2, 2, 'grip0')
end

end
constrained motion tried: True
result: 24 - (0, 1, 'grip0')->(0, 2, 'grip0') = success
branching: 24->29 (8.1/30.0 s, steps/err: 80(394.105911255 ms)/1.0667759317e-05)
try: 29 - (0, 2, 'grip0')->(1, 2, 'grip0')
result: 29 - (0, 2, 'grip0')->(1, 2, 'grip0') = success
branching: 29->30 (8.1/30.0 s, steps/err: 1(3.58819961548 ms)/0)
try: 30 - (1, 2, 'grip0')->(2, 2, 'grip0')
end
constrained motion tried: True
x
Goal reached
result: 30 - (1, 2, 'grip0')->(2, 2, 'grip0') = success
branching: 30->31 (8.48/30.0 s, steps/err: 81(375.394821167 ms)/0.000148197474697)
++ adding return motion to acquired answer ++
Goal reached
======================= terminated 0: required answers acquired  (8.8/30.0) ===============================
========================== FINISHED (8.8 / 30.0 s) ==============================]


In [13]:
snode_schedule = tplan.get_best_schedule(at_home=False)
ppline.play_schedule(snode_schedule)

(0, 0, 'knob')->(0, 0, 'grip0')
(0, 0, 'grip0')->(0, 1, 'grip0')
(0, 1, 'grip0')->(0, 2, 'grip0')
(0, 2, 'grip0')->(1, 2, 'grip0')
(1, 2, 'grip0')->(2, 2, 'grip0')


In [27]:
# for i_s, snode in sorted(tplan.snode_dict.items()):
snode = tplan.snode_dict[3]
pscene.set_object_state(snode.state)
gscene.show_pose(snode.state.Q)

In [28]:
# for i_s, snode in sorted(tplan.snode_dict.items()):
snode = tplan.snode_dict[6]
pscene.set_object_state(snode.state)
gscene.show_pose(snode.state.Q)

In [29]:
tplan.snode_dict[5].state.node

(0, 0, 'knob')

In [30]:
tplan.snode_dict[6].state.node

(0, 2, 'grip0')

In [25]:
snode.state.node

(0, 2, 'grip0')

In [26]:
snode.state.node

(0, 2, 'grip0')

In [ ]:
for snode in snode_schedule:
    print(snode.idx)

In [21]:
tplan.node_snode_dict[(1, 2, 'grip0')]

[9]

In [76]:
ppline.play_schedule(snode_schedule[-3:])

IndexError: list index out of range

In [90]:
snode_pre = tplan.snode_dict[3]

In [91]:
# snode_pre = snode_schedule[-2]

In [93]:
new_node = (1, 2,'grip0')

In [94]:
pscene.set_object_state(snode_pre.state)
gscene.show_pose(snode_pre.state.Q)

In [95]:
# for _ in range(100):
#     pscene.set_object_state(snode_pre.state)
#     gscene.show_pose(snode_pre.state.Q)
#     time.sleep(0.01)

In [96]:
# gscene.show_motion(snode.traj)

In [97]:
# Traj, Q, err, succ = plan_algorithm(mplan, snode_pre.state, snode.state, subject_list=subjects, verbose=True)

In [98]:
for _ in range(1):
    available_binding_dict = pscene.get_available_binding_dict(snode_pre.state, to_node=new_node)
    to_state = pscene.sample_leaf_state(snode_pre.state, available_binding_dict, new_node)
    subjects, _ = pscene.get_changing_subjects(snode_pre.state, to_state)
    print(subjects)
    gscene.update_markers_all()

['door']


In [99]:
Traj, end_state, err, succ = ppline.test_connection(from_state=snode_pre.state, to_state=to_state)
print(succ)

True


In [100]:
pscene.set_object_state(snode_pre.state)

In [101]:
gscene.show_motion(Traj)

In [24]:
# gscene.show_motion(Traj)

In [25]:
self, from_state, to_state, subject_list = mplan, snode_pre.state, to_state, subjects
timeout=1
timeout_joint=None
timeout_constrained=None
verbose=False
only_self_collision=False
kwargs = {}

In [108]:
np.round(to_state.binding_state["door"].T_actor_lh, 3)

array([[ 0.075, -0.705,  0.705, -0.204],
       [ 0.001, -0.707, -0.707, -0.247],
       [ 0.997,  0.053, -0.053,  0.159],
       [ 0.   ,  0.   ,  0.   ,  1.   ]], dtype=float32)

In [109]:
timeout_joint = timeout_joint if timeout_joint is not None else timeout
timeout_constrained = timeout_constrained if timeout_constrained is not None else timeout
self.planner.clear_context_cache()
self.planner.clear_manifolds()
if self.enable_dual:
    for dual_planner in self.dual_planner_dict.values():
        dual_planner.planner.clear_context_cache()
        dual_planner.planner.clear_manifolds()

if len(subject_list)>1:
    raise(RuntimeError("Only single manipulator operation is implemented with moveit!"))

self.update_gscene(only_self_collision=only_self_collision)

motion_type = 0

In [110]:
motion_type = MoveitPlanner.TASK_MOTION
obj_name, ap_name, binder_name, binder_geometry_name = to_state.binding_state[subject_list[0]].get_chain()

binder = self.pscene.actor_dict[binder_name]
obj = self.pscene.subject_dict[obj_name]
handle = obj.action_points_dict[ap_name]

group_name_handle = self.binder_link_robot_dict[handle.geometry.link_name] if handle.geometry.link_name in self.binder_link_robot_dict else None
group_name_binder = self.binder_link_robot_dict[binder.geometry.link_name] if binder.geometry.link_name in self.binder_link_robot_dict else None

btf = to_state.binding_state[obj_name]
T_handle = np.copy(btf.T_handle_lh)
T_binder = np.copy(btf.T_actor_lh)
T_add_handle = np.copy(btf.T_add_handle)
T_add_actor = np.copy(btf.T_add_actor)

In [111]:
dual = False
if group_name_binder and not group_name_handle:
    group_name = group_name_binder
    tool, T_tool = binder, T_binder
    target, T_tar = handle, T_handle
    T_add_tool = T_add_actor
    T_add_tar = T_add_handle
elif group_name_handle and not group_name_binder:
    group_name = group_name_handle
    tool, T_tool = handle, T_handle
    target, T_tar = binder, T_binder
    T_add_tool = T_add_handle
    T_add_tar = T_add_actor
elif group_name_handle == group_name_binder:
    if verbose:
        print("link transfer between same links: {} - {}".format(subject_list, group_name_binder))
#     return [from_state.Q], from_state.Q, 1, False
else:
    if not self.enable_dual:
        raise(RuntimeError("dual arm motion is not enabled - {} & {} - {}".format(
            group_name_handle, group_name_binder, subject_list)))
    dual = True
    group_name = "{}_{}".format(group_name_binder, group_name_handle)
    self.update_gscene(group_name, only_self_collision=only_self_collision)
    tool, T_tool = handle, T_handle
    target, T_tar = binder, T_binder
    T_add_tool = T_add_handle
    T_add_tar = T_add_actor

T_tar_tool = np.matmul(T_tar, SE3_inv(T_tool))
goal_pose = tuple(T_tar_tool[:3,3]) \
            +tuple(Rotation.from_dcm(T_tar_tool[:3,:3]).as_quat())
T_tar_tool_cur = self.gscene.get_tf(Q=from_state.Q, to_link=tool.geometry.link_name, from_link=target.geometry.link_name)

In [112]:
gscene.add_highlight_axis("hl", "ttt", T=T_tar_tool, link_name=target.geometry.link_name)

In [113]:
gscene.add_highlight_axis("hl", "tttc", T=T_tar_tool_cur, link_name=target.geometry.link_name)

In [114]:
gscene.add_highlight_axis("hl", "tt", T=T_tar, link_name=target.geometry.link_name)

In [115]:
gscene.add_highlight_axis("hl", "ttl", T=T_tool, link_name=tool.geometry.link_name)

In [46]:
gscene.clear_highlight()

In [47]:
pscene.show_binding(btf)

In [116]:
np.linalg.norm(T_tar_tool - T_tar_tool_cur)

6.401283803795755e-06

In [32]:

def plan_algorithm(self, from_state, to_state, subject_list, timeout=1,
                   timeout_joint=None, timeout_constrained=None, verbose=False, only_self_collision=False, **kwargs):
    timeout_joint = timeout_joint if timeout_joint is not None else timeout
    timeout_constrained = timeout_constrained if timeout_constrained is not None else timeout
    self.planner.clear_context_cache()
    self.planner.clear_manifolds()
    if self.enable_dual:
        for dual_planner in self.dual_planner_dict.values():
            dual_planner.planner.clear_context_cache()
            dual_planner.planner.clear_manifolds()

    if len(subject_list)>1:
        raise(RuntimeError("Only single manipulator operation is implemented with moveit!"))

    self.update_gscene(only_self_collision=only_self_collision)

    motion_type = 0
    if len(subject_list) == 0: # joint motion case
        motion_type = MoveitPlanner.JOINT_MOTION
        if from_state.Q is None or to_state.Q is None:
            raise(RuntimeError("No motion goal is defined!"))
        diffs = np.abs(from_state.Q - to_state.Q)
        joint_groups =[]
        for rname in self.robot_names:
            idx_rbt = self.combined_robot.idx_dict[rname]
            if np.sum(diffs[idx_rbt])>1e-3:
                joint_groups.append(rname)
        if len(joint_groups)==0:
            return np.array([from_state.Q, to_state.Q]), to_state.Q, 0, True
        dual = False
        if len(joint_groups)==1:
            group_name = joint_groups[0]
            idx_rbt = self.combined_robot.idx_dict[group_name]
            planner = self.planner
            if self.need_mapping:
                from_Q = from_state.Q[self.idx_pscene_to_mpc]
                to_Q = to_state.Q[[idx for idx in self.idx_pscene_to_mpc if idx in idx_rbt]]
            else:
                from_Q = from_state.Q
                to_Q =  to_state.Q[idx_rbt]
        else:
            print("from Q: {}".format(np.round(from_state.Q, 2)))
            print("tar  Q: {}".format(np.round(to_state.Q, 2)))
            raise(RuntimeError("multi-robot joint motion not implemented!"))
        if verbose:
            print("try joint motion") ## <- DO NOT REMOVE THIS: helps multi-process issue with boost python-cpp
        trajectory, success = planner.plan_joint_motion_py(
            group_name, tuple(to_Q), tuple(from_Q), timeout=timeout_joint, **kwargs)
        if success:
            trajectory = np.concatenate([trajectory, [to_state.Q]], axis=0)
        if verbose:
            print("joint motion tried: {}".format(success)) ## <- DO NOT REMOVE THIS: helps multi-process issue with boost python-cpp

    else: # task motion case
        motion_type = MoveitPlanner.TASK_MOTION
        obj_name, ap_name, binder_name, binder_geometry_name = to_state.binding_state[subject_list[0]].get_chain()

        binder = self.pscene.actor_dict[binder_name]
        obj = self.pscene.subject_dict[obj_name]
        handle = obj.action_points_dict[ap_name]

        group_name_handle = self.binder_link_robot_dict[handle.geometry.link_name] if handle.geometry.link_name in self.binder_link_robot_dict else None
        group_name_binder = self.binder_link_robot_dict[binder.geometry.link_name] if binder.geometry.link_name in self.binder_link_robot_dict else None

        btf = to_state.binding_state[obj_name]
        T_handle = btf.T_handle_lh
        T_binder = btf.T_actor_lh
        T_add_handle = btf.T_add_handle
        T_add_actor = btf.T_add_actor

        dual = False
        if group_name_binder and not group_name_handle:
            group_name = group_name_binder
            tool, T_tool = binder, T_binder
            target, T_tar = handle, T_handle
            T_add_tool = T_add_actor
            T_add_tar = T_add_handle
        elif group_name_handle and not group_name_binder:
            group_name = group_name_handle
            tool, T_tool = handle, T_handle
            target, T_tar = binder, T_binder
            T_add_tool = T_add_handle
            T_add_tar = T_add_actor
        elif group_name_handle == group_name_binder:
            if verbose:
                print("link transfer between same links: {} - {}".format(subject_list, group_name_binder))
            return [from_state.Q], from_state.Q, 1, False
        else:
            if not self.enable_dual:
                raise(RuntimeError("dual arm motion is not enabled - {} & {} - {}".format(
                    group_name_handle, group_name_binder, subject_list)))
            dual = True
            group_name = "{}_{}".format(group_name_binder, group_name_handle)
            self.update_gscene(group_name, only_self_collision=only_self_collision)
            tool, T_tool = handle, T_handle
            target, T_tar = binder, T_binder
            T_add_tool = T_add_handle
            T_add_tar = T_add_actor

        T_tar_tool = np.matmul(T_tar, SE3_inv(T_tool))
        goal_pose = tuple(T_tar_tool[:3,3]) \
                    +tuple(Rotation.from_dcm(T_tar_tool[:3,:3]).as_quat())
        T_tar_tool_cur = self.gscene.get_tf(Q=from_state.Q, to_link=tool.geometry.link_name, from_link=target.geometry.link_name)
        if np.linalg.norm(T_tar_tool - T_tar_tool_cur) < 1e-3:
            return [from_state.Q], from_state.Q, 0, True
        else:
            plan_algorithm.link_tar = target.geometry.link_name
            plan_algorithm.link_tool = tool.geometry.link_name
            plan_algorithm.T_tar_tool = T_tar_tool
            plan_algorithm.T_tar_tool_cur = T_tar_tool_cur
            print("err: {}".format(np.linalg.norm(T_tar_tool - T_tar_tool_cur)))

        if dual:
            dual_planner = self.dual_planner_dict[group_name]
            planner = dual_planner.planner
            from_Q = from_state.Q[dual_planner.idx_pscene_to_mpc]*dual_planner.joint_signs
        else:
            planner = self.planner
            if self.need_mapping:
                from_Q = from_state.Q[self.idx_pscene_to_mpc]
            else:
                from_Q = from_state.Q

        btf_from = from_state.binding_state[obj_name]
        btf_to = to_state.binding_state[obj_name]
        constraints = obj.make_constraints(btf_from.get_chain(),
                                           btf_to.get_chain())
        ref_link = self.chain_dict[group_name]["link_names"][0]
        Tref = self.gscene.get_tf(Q=from_Q, to_link=target.geometry.link_name, from_link=ref_link)
        if constraints:

            if self.incremental_constraint_motion:
                # ################################# Special planner ##############################
                T_ref_tool = matmul_series(Tref, T_tar_tool, tool.geometry.Toff)
                trajectory, success = self.get_incremental_traj(tool.geometry, T_ref_tool,
                                                                from_Q, step_size=0.01, ERROR_CUT=0.01,
                                                                SINGULARITY_CUT=0.01, VERBOSE=verbose,
                                                                ref_link=ref_link, VISUALIZE=self.visualize_increments)
                if self.debug_iterative_motion:
                    self.trajectory = trajectory
                    self.gscene.show_motion(trajectory)
            else:
                for motion_constraint in constraints:
                    self.add_constraint(group_name, tool.geometry.link_name, tool.Toff_lh, motion_constraint=motion_constraint)
                if verbose:
                    print("try constrained motion") ## <- DO NOT REMOVE THIS: helps multi-process issue with boost python-cpp
                ################################ Original planner ##############################
                trajectory, success = planner.plan_constrained_py(
                    group_name, tool.geometry.link_name, goal_pose, target.geometry.link_name, tuple(from_Q),
                    timeout=timeout_constrained, **kwargs)
            ################################################################################
            if verbose:
                print("constrained motion tried: {}".format(success)) ## <- DO NOT REMOVE THIS: helps multi-process issue with boost python-cpp
        else:
            replan_joint = False

            if verbose:
                print("try transition motion") ## <- DO NOT REMOVE THIS: helps multi-process issue with boost python-cpp

            pqr_pass = False
            if self.enable_PRQ:
                T_rtar = self.gscene.get_tf(target.geometry.link_name, from_Q,
                                            from_link=self.chain_dict[group_name]['link_names'][0])
                Tre = np.matmul(T_rtar, T_tar_tool)
                Q = self.sample_PRQ(group_name, Tre, radii=self.radii)
                if Q is not None:
                    from_Q_tmp = np.copy(from_Q)
                    from_Q_tmp[self.combined_robot.idx_dict[group_name]] = Q
                    ref_link = self.chain_dict[group_name]["link_names"][0]
                    Tref = self.gscene.get_tf(Q=from_Q_tmp, to_link=target.geometry.link_name, from_link=ref_link)
                    T_ref_tool = matmul_series(Tref, T_tar_tool, tool.geometry.Toff)
                    trajectory, success = self.get_incremental_traj(tool.geometry, T_ref_tool,
                                                               from_Q_tmp, step_size=0.01, ERROR_CUT=0.01,
                                                               SINGULARITY_CUT=0.01,
                                                               VERBOSE=verbose,
                                                               ref_link=ref_link,
                                                               VISUALIZE=self.visualize_increments)
                    if success:
                        to_Q = trajectory[-1]
                        if self.validate_trajectory([to_Q], update_gscene=False):
                            pqr_pass = True # pass plan_py, it will fail anyway if joint motion fails
                            if verbose:
                                print("[MPLAN] use PRQ")
                            pqr_kwargs = copy.deepcopy(kwargs)
                            pqr_kwargs.update(self.pqr_kwargs)
                            if "timeout" not in kwargs:
                                pqr_kwargs["timeout"] = timeout_joint
                            to_Q = to_Q[self.combined_robot.idx_dict[group_name]]
                            trajectory, success = self.planner.plan_joint_motion_py(
                                group_name, tuple(to_Q), tuple(from_Q), **pqr_kwargs)
                            if verbose and not success:
                                print("[MPLAN] PRQ joint motion failed")
                        elif verbose:
                            print("[MPLAN] PRQ initial not valid")
                    elif verbose:
                        print("[MPLAN] PRQ-target approach failed")
                elif verbose:
                    print("[MPLAN] PRQ not sampled")
            if not pqr_pass:
                trajectory, success = self.planner.plan_py(
                    group_name, tool.geometry.link_name, goal_pose, target.geometry.link_name, tuple(from_Q),
                    timeout=timeout, **kwargs)

            if verbose:
                print("transition motion tried: {}".format(success)) ## <- DO NOT REMOVE THIS: helps multi-process issue with boost python-cpp

    if success:
        if dual:
            trajectory = (trajectory*dual_planner.joint_signs)[:,dual_planner.idx_mpc_to_pscene]
        else:
            if self.need_mapping:
                trajectory = trajectory[:,self.idx_mpc_to_pscene]
            for rname in self.combined_robot.robot_names:
                if rname != group_name: # fix non-manipulating arm - projection in constrained motion sometimes generates motion in non-using arm
                    trajectory[:, self.combined_robot.idx_dict[rname]] = \
                        from_state.Q[self.combined_robot.idx_dict[rname]]
        Q_last = trajectory[-1]
        Q_last_dict = list2dict(Q_last, self.joint_names)
        if motion_type == MoveitPlanner.JOINT_MOTION:
            error = np.sum(np.abs(to_state.Q - Q_last))
        elif motion_type == MoveitPlanner.TASK_MOTION:
            T_tar, T_tool = target.get_tf_handle(Q_last_dict), tool.get_tf_handle(Q_last_dict)
            T_tar = np.matmul(T_tar, T_add_tar)
            T_tool = np.matmul(T_tool, T_add_tool)

            # T_handle = np.matmul(handle.Toff_lh, SE3(Rot_rpy(rpy_add_handle), point_add_handle))
            # T_binder = np.matmul(binder.Toff_lh, SE3(Rot_rpy(rpy_add_binder), point_add_binder))
            error = np.linalg.norm(T_tar-T_tool)
    else:
        Q_last, error = [0]*self.planner.joint_num, None
    return trajectory, Q_last, error, success

In [ ]:
snode

In [17]:
snode = tplan.snode_dict[tplan.node_snode_dict[(0,2,'grip0')][0]]

In [18]:
from_state = snode.state

In [19]:
pscene.set_object_state(from_state)
gscene.show_pose(from_state.Q)

In [21]:
tplan.node_parent_dict

{(0, 0, 'grip0'): {(0, 0, 'knob'), (0, 1, 'grip0'), (1, 0, 'grip0')},
 (0, 0, 'knob'): {(0, 0, 'grip0'), (0, 1, 'knob'), (1, 0, 'knob')},
 (0, 1, 'grip0'): {(0, 0, 'grip0'),
  (0, 1, 'knob'),
  (0, 2, 'grip0'),
  (1, 1, 'grip0')},
 (0, 1, 'knob'): {(0, 0, 'knob'),
  (0, 1, 'grip0'),
  (0, 2, 'knob'),
  (1, 1, 'knob')},
 (0, 2, 'grip0'): {(0, 1, 'grip0'), (0, 2, 'knob'), (1, 2, 'grip0')},
 (0, 2, 'knob'): {(0, 1, 'knob'), (0, 2, 'grip0'), (1, 2, 'knob')},
 (1, 0, 'grip0'): {(0, 0, 'grip0'),
  (1, 0, 'knob'),
  (1, 1, 'grip0'),
  (2, 0, 'grip0')},
 (1, 0, 'knob'): {(0, 0, 'knob'),
  (1, 0, 'grip0'),
  (1, 1, 'knob'),
  (2, 0, 'knob')},
 (1, 1, 'grip0'): {(0, 1, 'grip0'),
  (1, 0, 'grip0'),
  (1, 1, 'knob'),
  (1, 2, 'grip0'),
  (2, 1, 'grip0')},
 (1, 1, 'knob'): {(0, 1, 'knob'),
  (1, 0, 'knob'),
  (1, 1, 'grip0'),
  (1, 2, 'knob'),
  (2, 1, 'knob')},
 (1, 2, 'grip0'): {(0, 2, 'grip0'), (1, 1, 'grip0'), (1, 2, 'knob')},
 (1, 2, 'knob'): {(0, 2, 'knob'),
  (1, 1, 'knob'),
  (1, 2, 'grip0'

In [46]:
knob_s = pscene.subject_dict['knob']

In [47]:
knob_s.geometry

'base_link'

In [48]:
btf = from_state.binding_state['knob']

In [49]:
btf.binding

BindingChain(subject_name='knob', handle_name='r2', actor_name='knob_plug', actor_root_gname='lever')

In [52]:
door_hinge = pscene.actor_dict['door_hinge']

In [56]:
door_hinge.check_available(list2dict(from_state.Q, gscene.joint_names))

False

In [57]:
door_hinge.geometry.is_controlled()

False

In [59]:
door_hinge.geometry.link_name

'base_link'

### push back

In [18]:
gscene.show_pose(crob.home_pose)
term_state = snode_schedule[-1].state
print(pscene.subject_name_list)
print(term_state.node)

['knob', 'lever_grip']
(2, 'knob')


In [19]:
goal_nodes = [(1,'knob')]
ppline.search(term_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 36/36 agents
try: 0 - (2, 'knob')->(2, 'knob')
try: 0 - (2, 'knob')->(2, 'knob')
try: 0 - (2, 'knob')->(2, 'knob')
result: 0 - (2, 'knob')->(1, 'knob') = fail
try: 0 - (2, 'knob')->(2, 'grip0')
result: 0 - (2, 'knob')->(1, 'knob') = fail
result: 0 - (2, 'knob')->(1, 'knob') = fail
try transition motion
try: 0 - (2, 'knob')->(2, 'grip0')
try: 0 - (2, 'knob')->(2, 'grip0')
try transition motion
try transition motion
try: 0 - (2, 'knob')->(2, 'grip0')
try transition motion
try: 0 - (2, 'knob')->(2, 'grip0')
try: 0 - (2, 'knob')->(2, 'grip0')
transition motion tried: True
try: 0 - (2, 'knob')->(2, 'grip0')
try: 0 - (2, 'knob')->(2, 'grip0')
try: 0 - (2, 'knob')->(2, 'grip0')
result: 0 - (2, 'knob')->(2, 'grip0') = success
try transition motion
try: 0 - (2, 'knob')->(2, 'grip0')
try: 0 - (2, 'knob')->(2, 'knob')
ERROR sampling parent from : (2, 'knob') / parent nodes: set([(2, 'grip0')])
try transition motion
try transition motion
try: 0 - (2, 'knob')->(2, 'knob')
transition motion trie

try: 10 - (2, 'knob')->(2, 'knob')
try: 20 - (2, 'knob')->(2, 'knob')
try: 0 - (2, 'knob')->(2, 'knob')
try: 2 - (2, 'grip0')->(2, 'grip0')
try: 20 - (2, 'knob')->(2, 'grip0')
result: 11 - (2, 'knob')->(1, 'knob') = fail
branching: 4->18 (0.67/30.0 s, steps/err: 7(92.8030014038 ms)/0.00193547850729)
try: 20 - (2, 'knob')->(2, 'grip0')
result: 20 - (2, 'knob')->(1, 'knob') = fail
result: 0 - (2, 'knob')->(1, 'knob') = fail
try: 18 - (2, 'knob')->(2, 'grip0')
result: 10 - (2, 'knob')->(1, 'knob') = fail
result: 0 - (2, 'knob')->(2, 'grip0') = success
try transition motion
try: 11 - (2, 'knob')->(2, 'knob')
try transition motion
try constrained motion
try: 16 - (2, 'grip0')->(2, 'grip0')
try transition motion
try: 0 - (2, 'knob')->(2, 'grip0')
branching: 0->21 (0.72/30.0 s, steps/err: 51(466.411113739 ms)/0.000981844614581)
try constrained motion
try transition motion
try: 12 - (2, 'knob')->(2, 'knob')
result: 12 - (2, 'knob')->(1, 'knob') = fail
result: 11 - (2, 'knob')->(1, 'knob') = fa

result: 10 - (2, 'knob')->(1, 'knob') = fail
branching: 20->36 (1.04/30.0 s, steps/err: 6(116.895914078 ms)/0.00139235918776)
try transition motion
try: 14 - (2, 'knob')->(2, 'grip0')
try: 2 - (2, 'grip0')->(2, 'grip0')
try: 21 - (2, 'grip0')->(2, 'grip0')
try transition motion
try constrained motion
try constrained motion
try: 31 - (2, 'knob')->(2, 'grip0')
transition motion tried: True
joint max
result: 19 - (2, 'grip0')->(2, 'knob') = success
constrained motion tried: False
branching: 19->37 (1.1/30.0 s, steps/err: 9(71.624994278 ms)/0.00175082406999)
Motion Plan Failure
try transition motion
transition motion tried: True
result: 21 - (2, 'grip0')->(1, 'grip0') = fail
try: 37 - (2, 'knob')->(2, 'knob')
try: 18 - (2, 'knob')->(2, 'knob')
transition motion tried: True
transition motion tried: True
result: 17 - (2, 'grip0')->(2, 'knob') = success
result: 14 - (2, 'knob')->(2, 'grip0') = success
transition motion tried: True
result: 22 - (2, 'knob')->(2, 'grip0') = success
branching: 22

try: 55 - (1, 'grip0')->(1, 'knob')
end
end
result: 54 - (2, 'knob')->(1, 'knob') = fail
constrained motion tried: True
Goal reached
result: 4 - (2, 'grip0')->(1, 'grip0') = success
end
constrained motion tried: True
branching: 15->55 (1.49/30.0 s, steps/err: 80(821.505069733 ms)/2.16168044012e-05)
result: 4 - (2, 'grip0')->(1, 'grip0') = success
try: 28 - (2, 'grip0')->(2, 'grip0')
constrained motion tried: True
result: 2 - (2, 'grip0')->(1, 'grip0') = success
branching: 4->57 (1.5/30.0 s, steps/err: 80(983.791828156 ms)/2.92268849744e-06)
try constrained motion
branching: 4->58 (1.52/30.0 s, steps/err: 80(999.078989029 ms)/2.92268849744e-06)
branching: 2->59 (1.52/30.0 s, steps/err: 80(811.466217041 ms)/5.0228753776e-06)
result: 55 - (1, 'grip0')->(1, 'knob') = success
branching: 55->56 (1.53/30.0 s, steps/err: 1(6.05702400208 ms)/0)
try: 57 - (1, 'grip0')->(1, 'knob')
try transition motion
++ adding return motion to acquired answer ++
try: 50 - (2, 'knob')->(2, 'grip0')
try: 59 - (1

======================= terminated 34: Stop called from other agent  (1.9/30.0) ===============================
branching: 44->72 (1.86/30.0 s, steps/err: 2(129.889011383 ms)/0.00112631813329)
======================= terminated 6: Stop called from other agent  (1.9/30.0) ===============================
transition motion tried: True
======================= terminated 35: Stop called from other agent  (1.9/30.0) ===============================
======================= terminated 9: Stop called from other agent  (1.9/30.0) ===============================
======================= terminated 28: Stop called from other agent  (1.9/30.0) ===============================
======================= terminated 4: required answers acquired  (1.9/30.0) ===============================
Goal reached
result: 65 - (1, 'grip0')->(1, 'knob') = success
transition motion tried: True
======================= terminated 26: Stop called from other agent  (1.9/30.0) ===============================
branching: 65->73 (

end
constrained motion tried: True
end
result: 13 - (2, 'grip0')->(1, 'grip0') = success
constrained motion tried: True
branching: 13->85 (2.17/30.0 s, steps/err: 80(716.837882996 ms)/8.1243187965e-06)
result: 47 - (1, 'grip0')->(2, 'grip0') = success
======================= terminated 3: Stop called from other agent  (2.2/30.0) ===============================
end
constrained motion tried: True
end
branching: 47->86 (2.17/30.0 s, steps/err: 80(711.130857468 ms)/2.87955654637e-06)
constrained motion tried: True
result: 1 - (2, 'grip0')->(1, 'grip0') = success
result: 19 - (2, 'grip0')->(1, 'grip0') = success
======================= terminated 18: Stop called from other agent  (2.2/30.0) ===============================
branching: 19->88 (2.2/30.0 s, steps/err: 80(1013.09490204 ms)/5.70565958244e-06)
branching: 1->87 (2.19/30.0 s, steps/err: 80(745.685815811 ms)/1.05078210319e-05)
======================= terminated 5: required answers acquired  (2.2/30.0) ===============================
=

In [20]:
snode_schedule = tplan.get_best_schedule()
ppline.play_schedule(snode_schedule)

(2, 'knob')->(2, 'grip0')
(2, 'grip0')->(1, 'grip0')
(1, 'grip0')->(1, 'knob')
(1, 'knob')->(1, 'knob')
